In [120]:
import pandas as pd
import numpy as np
import itertools as it
import networkx as nx
# Documentacion de la libreria: http://networkx.readthedocs.io/en/networkx-1.11/

In [121]:
def filterData(df, isTraining, date):
    """
        Funcion que devuelve el conjunto de problemas que tienen status AC o PE
        Si isTraining es true, entonces la funcion sacara el training_set, si no, sacara el evaluation_set
        date es la fecha de particion
    """
    
    if isTraining:
        df = df[df['submissionDate'] < date]
        df = df.loc[df['status'].isin(['AC', 'PE'])]
    else:
        df = df[df['submissionDate'] >= date]
    
    

    return df

In [122]:
def compareNodes(f_list, s_list):
    """
        Funcion que devuelve el numero de usuarios que han hecho ambos problemas
    """
    peso = len(np.intersect1d(f_list, s_list))
    
    return peso
    
def createLinks(prob_us_set, nodos):
    """
        Funcion que crea los enlaces del grafo a partir de la informacion contenida en el conjunto que se le
        pasa a la funcion
    """
    resultado = list() 
    
    # hago todas las posibles combinaciones de problemas
    for fst, snd in it.combinations(nodos, 2):
        # obtengo el peso pasando la lista de usuarios que ha hecho cada problema
        peso = compareNodes(prob_us_set[fst], prob_us_set[snd])
        if peso >= 1:
            resultado.append((fst, snd, peso))
            
            
            
    return resultado

In [123]:
def filterWeight(weightUmbral, linksToFilter):
    """
        Funcion que filtra los enlaces de un grafo, para que el peso sea mayor o igual al dado
    """
    
    result = [(x, y, z) for (x, y, z) in linksToFilter if z >= weightUmbral]
    
    return result
    

In [124]:
def create_graph_nx(list_nodes, list_links):
    """
        Funcion que crea un grafo de tipo Graph de la libreria NetworkX
        Construccion del grafo: http://networkx.readthedocs.io/en/networkx-1.11/tutorial/tutorial.html#what-to-use-as-nodes-and-edges
    """
    grafo = nx.Graph() # creo la variable grafo

    # incluyo los nodos del grafo 
    grafo.add_nodes_from(list_nodes)

    # se incluyen las tuplas de enlaces con el peso del enlace
    # es una lista de la forma [(Nodo1, Nodo2, peso), ......]
    grafo.add_weighted_edges_from(list_links)

    return grafo

In [125]:
# MAIN
# ---------

# se guarda en la variable df (DataFrame) toda la base de datos
df = pd.read_csv('bbdd_orderbydate.csv')

# aqui quito los problemas que no existian despues de la fecha umbral
df = df[df['problem_id'] <= 511] 

# construyo el conjunto de entrenamiento
training_set = filterData(df, True, "2016-10-21 00:00:00")

print(len(training_set))

# obtengo los nodos del grafo:
nodes = training_set.problem_id.unique()


# creo un diccionario que va a tener a los problemas como keys y los valores seran los
# usuarios que han hecho ese problema
grouped = training_set.groupby('problem_id')['user_id'].apply(list)

print(len(nodes))
# print(grouped)


10262
169


In [126]:
print(training_set)

       problem_id  user_id status       submissionDate
0              10        5     AC  2014-02-17 15:27:07
1               2        6     AC  2014-02-17 15:39:17
2               2        9     AC  2014-02-18 00:30:14
3              10        9     AC  2014-02-18 00:34:46
4               4        9     AC  2014-02-18 00:50:28
5               6        9     AC  2014-02-18 00:52:11
6              13        9     AC  2014-02-18 00:53:40
9              15        8     AC  2014-02-19 19:58:03
10              4        8     AC  2014-02-20 14:23:30
11             39       16     AC  2014-02-20 15:44:33
12             39       17     AC  2014-02-20 16:53:34
13             13       12     AC  2014-02-21 11:08:38
15             33       12     AC  2014-02-21 11:58:32
16             39       12     AC  2014-02-21 12:05:49
17             44       12     AC  2014-02-21 12:52:44
18             44       15     AC  2014-02-21 14:51:52
19             44       22     AC  2014-02-21 14:58:55
21        

In [127]:
# OBTENCION DEL EVALUATION_SET
# -------

# ahota saco el evaluation_set
evaluation_set = filterData(df, False, "2016-10-21 00:00:00")

print(evaluation_set)

# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user_eval = evaluation_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users_eval = pd.DataFrame({'user_id':grouped_user_eval.index, 'list_problem_id':grouped_user_eval.values})

print(df_users_eval)

       problem_id  user_id status       submissionDate
16939         469      799     RF  2016-10-21 00:08:23
16940         325     3832     AC  2016-10-21 00:08:35
16942         469     3757     AC  2016-10-21 00:41:36
16943         469     3792    RTE  2016-10-21 01:04:08
16944         469      810     AC  2016-10-21 01:48:35
16945         438     3943     AC  2016-10-21 02:23:56
16946         469     3773     AC  2016-10-21 05:00:58
16947         346     3728     AC  2016-10-21 09:06:38
16948         469     3736     CE  2016-10-21 09:38:14
16949         438     3785     TL  2016-10-21 09:43:52
16950         469     3704     AC  2016-10-21 10:04:03
16951         469     2912     AC  2016-10-21 10:20:36
16952         213     3136     AC  2016-10-21 10:28:23
16953         213     3088     AC  2016-10-21 10:29:26
16954         469     3708    RTE  2016-10-21 10:35:15
16956         235     3136    RTE  2016-10-21 10:37:58
16957         183     3880     AC  2016-10-21 11:38:11
16958     

In [128]:

# creo los enlaces a partir de la informacion de los nodos
links = createLinks(grouped, nodes)
# ahora filtro el grafo para que los enlaces solo tengan el peso que quiero
linksFiltered = filterWeight(5, links)

print(len(linksFiltered))

# aqui creo el grafo 
graph = create_graph_nx(nodes, linksFiltered)

10837


In [129]:
def apply_aa(row, graph):
    """
        Funcion que devuelve para cada par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """
    
    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de AA para el par de nodos
    value = nx.adamic_adar_index(graph, [(row['one'], row['two'])])
    
    value_aa = 0
    for u, v, p in value:
        # itero el iterador, guardando el valor de adar adamic
        value_aa = p
    
    return value_aa

def create_aa_data(graph, nodes):

    # Ahora voy a construir un DataFrame que tenga dos columnas con todas las posibles combinaciones de problemas, y otra 
    # columna con el valor de aa para ese par de problemas
    fst_column = list()
    snd_column = list()
    for fst, snd in it.combinations(nodes, 2):
        fst_column.append(fst)
        snd_column.append(snd)

    d = {'one' : fst_column,
        'two' : snd_column}
    dataFrame_aa = pd.DataFrame(d)


    # Aplico la funcion a cada fila
    dataFrame_aa['aa'] = dataFrame_aa.apply (lambda row: apply_aa(row, graph), axis=1)


    return dataFrame_aa

In [130]:
aa_df = create_aa_data(graph, nodes)
print(aa_df)

       one  two         aa
0       10    2  29.481399
1       10    4  29.480613
2       10    6  29.480613
3       10   13  28.835311
4       10   15  28.794947
5       10   39  29.481399
6       10   33  29.273420
7       10   44  29.017307
8       10   19  28.171070
9       10   27  29.480877
10      10   60  24.280302
11      10  100  29.481399
12      10   93  10.584318
13      10   53  28.222100
14      10   51  28.411760
15      10   49  29.257845
16      10   70  28.210133
17      10   62  29.007155
18      10   35  28.373895
19      10   81  25.908110
20      10    8  26.367535
21      10   17  24.092536
22      10   23  28.165266
23      10   25   0.000000
24      10   86  29.481139
25      10   29  27.592215
26      10   31   0.000000
27      10   47   0.000000
28      10   65  28.794680
29      10   73  27.133130
...    ...  ...        ...
14166  510  509  28.831763
14167  510  507  20.839273
14168  511  504   0.000000
14169  511  503  29.041361
14170  511  505  28.417354
1

In [131]:
# creo un diccionario que va a tener a los usuarios como keys y a los problemas que ha hecho como valores
# a partir del conjunto de entrenamiento
grouped_user = training_set.groupby('user_id')['problem_id'].apply(list)

# convierto la serie en un dataframe
df_users = pd.DataFrame({'user_id':grouped_user.index, 'list_problem_id':grouped_user.values})

print(df_users)

                                        list_problem_id  user_id
0                                     [10, 76, 489, 39]        5
1                                              [2, 487]        6
2                      [39, 44, 76, 105, 109, 114, 117]        7
3     [15, 4, 19, 27, 60, 53, 105, 76, 83, 90, 147, ...        8
4          [2, 10, 4, 6, 13, 253, 57, 62, 143, 83, 109]        9
5                                             [307, 39]       10
6                           [33, 109, 39, 93, 100, 155]       11
7     [13, 33, 39, 44, 100, 51, 49, 70, 81, 53, 130,...       12
8                                         [44, 187, 39]       15
9                                                  [39]       16
10                                             [39, 86]       17
11                            [119, 114, 316, 507, 506]       18
12                                                 [44]       22
13                                            [100, 13]       24
14    [258, 141, 309, 310

In [132]:
def lenProblemsDone(row, set_filter):
    """
        Funcion auxiliar que calcula cuanto problemas ha hecho cada usuario en un conjunto: training o evaluation
    """
    # saco el dataframe que contendra solo una fila con la lista de problemas que ha hecho el usuario
    df_filter = set_filter[set_filter['user_id'] == row['user_id']]
    
    if df_filter.empty:
        # si esta vacio, entonces es que el usuario no ha hecho problemas en ese conjunto
        return 0
    else:
        # sino, devuelvo la longitud de la lista de problemas
        return len(df_filter['list_problem_id'].iloc[0]) 
    
    

In [133]:
# aqui voy a hacer el filtro de usuarios de forma que para hacer las recomendaciones solo tengamos en 
# cuenta aquellos usuarios que han hecho 5 o mas problemas tanto antes de la fecha limite como despues

# primero guardo la lista de usuarios
user_list = df.user_id.unique()

# la meto en un dataframe 
column_user_filter = {'user_id': user_list}
datraframe_user_filter = pd.DataFrame.from_dict(column_user_filter)


# ahora tengo que calcular para cada fila, el numero de problemas que han hecho en el training_set, evaluation_set
datraframe_user_filter['len_training'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users), axis=1)
datraframe_user_filter['len_evaluation'] = datraframe_user_filter.apply (lambda row: lenProblemsDone(row, df_users_eval), axis=1)
print(datraframe_user_filter)


      user_id  len_training  len_evaluation
0           5             4               1
1           6             2               0
2           9            11               3
3           8            47               0
4          16             1               0
5          17             2               2
6          12            12               0
7          18             5               8
8          15             3               1
9          22             1               0
10          7             7               0
11         27             0               0
12         28             1               0
13         29             1               0
14         32             2               0
15         35            41               1
16         33             9               0
17         39            11               0
18         31             9               0
19         40             1               0
20         44             8               0
21         48             0     

In [134]:
# ahora tengo que hacer el filtro en este dataframe, de forma que solo aparezcan las filas en las que len_training y 
# len_evaluation sea >=5
datraframe_user_filter = datraframe_user_filter[(datraframe_user_filter['len_training'] >= 5) & (datraframe_user_filter['len_evaluation'] >=5)]
print(datraframe_user_filter)

# aqui voy a guardar la lista de usuarios a los que voy a recomendar
user_list_to_recommend = sorted(datraframe_user_filter['user_id'].tolist())
print(user_list_to_recommend)
print(len(user_list_to_recommend))


      user_id  len_training  len_evaluation
7          18             5               8
29         60            18              14
118        25            22               7
261       130            80               6
299       414            49              61
316       443            53               8
317       448            43              21
483       689            33               7
619       912            13              11
633       935           127               9
1196     1711           128              16
1288     1893             7               6
1311     1952            19               9
1339     1955             9               5
1416     2038             5               5
1423     2096            12              13
1446     2051            11               8
1447     2025            12               8
1448     2120             7               8
1451     2041             9               7
1528     2257             6              13
1694     2576             5     

In [135]:
# ahora tengo que filtrar df_users para que solo contenga las filas en las que los usuarios
# pertenecen a la anterior lista

df_users = df_users[df_users['user_id'].isin(user_list_to_recommend)]
print(df_users)

                                        list_problem_id  user_id
11                            [119, 114, 316, 507, 506]       18
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711
997                      [39, 49, 465, 159, 134, 13, 2]     1893
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952
1024      [325, 327, 441, 468, 471, 503, 226, 109, 446]     1955
1066  [155, 505, 49, 162,

In [136]:
# creo el nuevo dataframe con los resultados 
column_problem_recomend = {'problem_id': nodes}
dataframe_problem_recomend = pd.DataFrame.from_dict(column_problem_recomend)

print(dataframe_problem_recomend)

     problem_id
0            10
1             2
2             4
3             6
4            13
5            15
6            39
7            33
8            44
9            19
10           27
11           60
12          100
13           93
14           53
15           51
16           49
17           70
18           62
19           35
20           81
21            8
22           17
23           23
24           25
25           86
26           29
27           31
28           47
29           65
..          ...
139         444
140         437
141         438
142         443
143         445
144         442
145         440
146         446
147         465
148         471
149         468
150         470
151         469
152         467
153         489
154         485
155         486
156         487
157         484
158         488
159         508
160         510
161         511
162         504
163         503
164         505
165         506
166         502
167         509
168         507

[169 ro

In [137]:
def getSimilarProblems(row, aa_df):
    """
        Funcion que devuelve la lista de los problemas similares del problema de la fila: aquellos que tienen un valor AA
        mayor a cero
    """
    # print(row['user_id'])
    
    # obtengo dos df con los usuarios que tienen usuarios vecinos con el usuario de la fila 
    column_result_one_tmp = aa_df[aa_df['one'] == row['problem_id']]
    column_result_one = column_result_one_tmp[column_result_one_tmp['aa'] > 0]
    column_result_two_tmp = aa_df[aa_df['two'] == row['problem_id']]
    column_result_two = column_result_two_tmp[column_result_two_tmp['aa'] > 0]
    
    # saco las listas de usuarios con usuarios comunes
    list_one = list(column_result_one['two'])
    list_two = list(column_result_two['one'])
    
    # la concateno sin tener en cuenta repeticiones, porque nunca va a haber
    list_aa = list_one + list_two
    
    # print(list_aa)
    
    if list_aa == []: # sino tiene vecinos en comun, pongo toda la lista de nodos
        list_aa = graph.nodes()
        list_aa.remove(row['problem_id']) # y elimino el nodo que estoy mirando
    
    # hago el filtro de los k mejores
    return list_aa

In [138]:
dataframe_problem_recomend['neighbors'] = dataframe_problem_recomend.apply (lambda row: getSimilarProblems(row, aa_df), axis=1)

# aqui tengo la lista de usuarios con sus k usuarios similares
print(dataframe_problem_recomend)

     problem_id                                          neighbors
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...
13           93  [53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86, 2

In [139]:
print(grouped)
# convierto la serie en un dataframe
df_users_recommend = pd.DataFrame({'problem_id':grouped.index, 'list_user_id':grouped.values})
print(df_users_recommend)

problem_id
2      [6, 9, 33, 67, 56, 84, 35, 60, 112, 124, 106, ...
4      [9, 8, 35, 84, 126, 44, 39, 313, 42, 437, 130,...
6      [9, 46, 35, 67, 89, 84, 137, 152, 231, 221, 88...
8      [35, 52, 133, 129, 130, 437, 511, 605, 935, 67...
10     [5, 9, 35, 133, 129, 80, 137, 247, 437, 103, 5...
13     [9, 12, 59, 35, 67, 109, 54, 24, 55, 106, 136,...
15     [8, 35, 67, 55, 133, 129, 84, 167, 205, 231, 2...
17           [35, 133, 129, 437, 103, 206, 62, 89, 1711]
19     [8, 35, 44, 205, 437, 527, 448, 1063, 206, 62,...
23     [35, 67, 60, 437, 206, 62, 103, 864, 1001, 113...
25                                        [35, 103, 130]
27     [8, 35, 67, 167, 39, 440, 448, 437, 130, 488, ...
29     [35, 133, 86, 112, 42, 232, 248, 247, 314, 320...
31                                        [35, 437, 448]
33     [12, 59, 35, 84, 111, 96, 152, 126, 155, 30, 1...
35     [64, 35, 214, 120, 236, 82, 409, 390, 175, 249...
39     [16, 17, 12, 7, 31, 40, 33, 66, 68, 71, 69, 70...
44     [12, 15, 22, 

In [140]:
def aa_value(one, two, graph):
    """
        Funcion que devuelve para ese par de nodos, el sumatorio de 1/log(N(z)), siendo N(z) el grado del nodo z para todo z 
        perteneciente al conjunto de nodos en comun de ese par de nodos
    """

    # obtengo un iterador de un solo elemento que tiene en la tercera posicion el valor de AA para el par de nodos
    value = nx.adamic_adar_index(graph, [(one, two)])
    # print(value)
    value_aa = -1
    for u, v, p in value:
        # print(p)
        # itero el iterador, guardando el valor de adar adamic
        value_aa = p

    return value_aa

In [141]:
def getWeights(row, graph):
    """
        Funcion que calcula la suma de todos los valores de aa de todos los nodos con los que tiene aa
    """
    
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # print(neighbors_list)
    
    problem = row['problem_id']
    suma = 0
    
    for elem in neighbors_list:
        # print(suma)
        # print(graph[user][elem]['weight'])
        suma = suma + aa_value(problem, elem, graph)
        
    return suma

In [142]:
def getPonderaciones(row, graph):
    """
        Funcion que calcula la ponderacion para cada problema vecino del de la fila
        Calculo la ponderacion diviendo el peso del enlace que enlaza cada problema con problem_id con la suma 
        total de los pesos
    """
    # primero obtengo la lista de los vecinos
    neighbors_list = row['neighbors']
    
    # obtengo la suma de pesos de los enlaces de esa lista
    total_weight = row['total_weight']
    
    # obtengo el id del usuario al que quiero recomendar
    problem = row['problem_id']
    
    lista_ponderaciones = list()
    
    for elem in neighbors_list:
        
        # obtengo el peso del enlace
        peso_enlace = aa_value(problem, elem, graph)
        
        if total_weight == 0:
            ponderacion = 0
        else:
            # hago la ponderacion
            ponderacion = peso_enlace/total_weight
        
        lista_ponderaciones.append(ponderacion)
        
    return lista_ponderaciones
    

In [143]:
# ahora voy a incluir una nueva columna que tenga las ponderaciones (suma de todos los enlaces) de cada usuario
dataframe_problem_recomend['total_weight'] = dataframe_problem_recomend.apply (lambda row: getWeights(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [144]:
# para cada usuario a "recomendar" para cada problema miro el numero de apariciones en su lista

dataframe_problem_recomend['score'] = dataframe_problem_recomend.apply (lambda row: getPonderaciones(row, graph), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [145]:
def getUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de la concatenacion de listas de usuarios que han
        realizado los problemas similares al de la fila. Además eliminara los usuarios que ya hayan realizado el problema.
    """
    
    # obtengo la lista de usuarios que han hecho el problema en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # print(list_problems_user)

    # lista resultante de la concatenacion de las listas de usuarios de los problemas similares
    list_result = list()
    
    # obtengo la longitud de la lista de vecinos de ese usuario
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de usuarios vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de usuarios que han hecho el problema similar
        list_problems_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_problems_df['list_user_id'])[0]
        
        # print("----------")
        #print(lista_problemas_comprobar)
        
        # aqui hago el filtro para que no se incluyan los usuarios que ya ha hechon el problema
        list_users = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora concateno el resultado
        list_result = list_result + list_users
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [146]:
# ahora para cada problema, hacer una lista de los usuarios similares a los que han realizado ese problema, 
# que no sean usuarios que han realizado ya ese problema
dataframe_problem_recomend['list_users'] = dataframe_problem_recomend.apply (lambda row: getUsersFromSimilarProblems(row, df_users_recommend), axis=1)
print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [147]:
def delRepetitions(row):
    """
        Funcion auxiliar para evitar que salgan repeticiones en las recomendaciones. Saco la lista de posibles 
        recomendaciones con valores unicos
    """
    conjunto_vacio = set()
    
    # esto sirve para que se haga mas rapido la comprobacion de si el elemento esta en la lista o no
    function_add = conjunto_vacio.add
    
    # hago la lista intensional, para mantener el orden dado en la lista original
    return [x for x in row['list_users'] if not (x in conjunto_vacio or function_add(x))]

In [148]:

# tengo que contar el numero de apariciones de cada usuario en la lista de usuarios

# voy a sacar primero una lista sin repeticiones
# ahora voy a crear una nueva columna que contenga la lista de usuarios sin repeticiones
dataframe_problem_recomend['lista_users_unique'] = dataframe_problem_recomend.apply(lambda row: delRepetitions(row), axis=1)

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [149]:
# elimino la columna de la lista de problemas (la que tiene repeticiones), ya que no interesa
del dataframe_problem_recomend['list_users']
# elimino la columna de total_weight, ya que no interesa
del dataframe_problem_recomend['total_weight']

print(dataframe_problem_recomend)

     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [150]:
def getListUsersFromSimilarProblems(row, df_users_recommend):
    """
        Funcion que va a devolver por cada fila una lista procedente de listas la concatenacion de listas de usuarios que han
        hecho los problemas similares a ese. Además eliminara los usuarios que ya hayan hecho el problema
    """
    
    # obtengo la lista de problemas que ha hecho el usuario en cuestion
    list_problems_users = df_users_recommend[df_users_recommend['problem_id'] == row['problem_id']]
    list_problems_user = list(list_problems_users['list_user_id'])[0]
   
    # lista resultante 
    list_result = list(list())
    
    # obtengo la longitud de la lista de vecinos de ese problema
    list_neighbors = row['neighbors']
    k = len(list_neighbors)
    
    # recorro la lista de problemas vecinos 
    for i in range(0, k):
        # print(row['list_similar_users'][i])
        # aqui saco la lista de problemas que ha hecho el usuario similar
        list_users_df = df_users_recommend[df_users_recommend['problem_id'] == row['neighbors'][i]]
        lista_problemas_comprobar = list(list_users_df['list_user_id'])[0]
        
        # aqui hago el filtro para que no se incluyan los problemas que ya ha hecho el usuario
        list_problems = [x for x in lista_problemas_comprobar if x not in list_problems_user]
        
        # ahora incluyo la lista en la lista
        list_result.append(list_problems)
        # print(list_problems)
        # print(list_result)
        # print("---------------")
    
    return list_result

In [151]:
dataframe_problem_recomend['lista_users_por_problema'] = dataframe_problem_recomend.apply(lambda row: getListUsersFromSimilarProblems(row, df_users_recommend), axis=1)

print(dataframe_problem_recomend)


     problem_id                                          neighbors  \
0            10  [2, 4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100,...   
1             2  [4, 6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93...   
2             4  [6, 13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 5...   
3             6  [13, 15, 39, 33, 44, 19, 27, 60, 100, 93, 53, ...   
4            13  [15, 39, 33, 44, 19, 27, 60, 100, 93, 53, 51, ...   
5            15  [39, 33, 44, 19, 27, 60, 100, 93, 53, 51, 49, ...   
6            39  [33, 44, 19, 27, 60, 100, 93, 53, 51, 49, 70, ...   
7            33  [44, 19, 27, 60, 100, 93, 53, 51, 49, 70, 62, ...   
8            44  [19, 27, 60, 100, 93, 53, 51, 49, 70, 62, 35, ...   
9            19  [27, 60, 100, 93, 53, 51, 49, 70, 62, 35, 81, ...   
10           27  [60, 100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 1...   
11           60  [100, 93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 2...   
12          100  [93, 53, 51, 49, 70, 62, 35, 81, 8, 17, 23, 86...   
13           93  [53

In [152]:
# ahora voy a separar cada problema_user para hacer la cuenta
# creo un nuevo dataframe que agrupa por el problema con su posible usuario
df_separation = dataframe_problem_recomend.groupby(['problem_id']).lista_users_unique.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)

df_separation.columns = ['problem_id', 'possible_user', ]

print(df_separation)

        problem_id  possible_user
0                2            126
1                2             39
2                2            313
3                2             42
4                2           1902
5                2           1983
6                2           1959
7                2           1988
8                2           1952
9                2           1962
10               2           3497
11               2             46
12               2            152
13               2            221
14               2             88
15               2            194
16               2             90
17               2             99
18               2            155
19               2            542
20               2            896
21               2           1186
22               2            598
23               2            584
24               2            596
25               2            594
26               2            593
27               2            581
28            

In [153]:
def getScoring(row, dataframe_problem_recomend):
    """
        Funcion en la que por cada usuario-problema, hace el scoring para ese problema (para ese usuario)
        sumando todas las ponderaciones de ese problema para ese usuario
    """
     # obtengo la lista de listas de usuarios que han hecho los vecinos del problema
    list_problems_users_df = dataframe_problem_recomend[dataframe_problem_recomend['problem_id'] == row['problem_id']]
    list_user_per_problem = list(list_problems_users_df['lista_users_por_problema'])[0]
    
    # saco tambien la lista de scores por vecinos
    list_scores_per_problem = list(list_problems_users_df['score'])[0]
    
    # print(list_scores_per_user)
    
    # obtengo el usuario del que quiero calcular el scoring
    user = row['possible_user']
    # print(problem)
    
    suma = 0
    k = len(list_user_per_problem)
    
    # por cada lista de la lista, saco la ponderacion de su correspondiente usuario (el usuario que lo ha hecho)
    # si problem aparece en la lista, sumo esa ponderacion
    for i in range(0, k):
        lista_a_comprobar = list_user_per_problem[i]
        # print(lista_a_comprobar)
        if user in lista_a_comprobar:
            # print(list_scores_per_user[i])
            suma = suma + list_scores_per_problem[i]
    
    return suma

In [154]:
# ahora voy a calcular el scoring para cada usuario
df_separation['score'] = df_separation.apply(lambda row: getScoring(row, dataframe_problem_recomend), axis=1)

print(df_separation)

        problem_id  possible_user     score
0                2            126  0.053626
1                2             39  0.073307
2                2            313  0.007003
3                2             42  0.157995
4                2           1902  0.013804
5                2           1983  0.232405
6                2           1959  0.137417
7                2           1988  0.088980
8                2           1952  0.128214
9                2           1962  0.344295
10               2           3497  0.048848
11               2             46  0.055763
12               2            152  0.103345
13               2            221  0.021040
14               2             88  0.014197
15               2            194  0.007003
16               2             90  0.027461
17               2             99  0.021337
18               2            155  0.042107
19               2            542  0.123496
20               2            896  0.062947
21               2           118

In [155]:
# ahora voy a ordenar en funcion del score de mayor a menor para cada usuario
# ahora lo que quiero es ordenar los problemas por cada usuario en funcion de su ponderacion
# primero ordeno por su valor de user y luego por el de ponderacion, de forma que quedan ordenador por su valor de ponderacion
df_separation = df_separation.sort_values(by=['possible_user', 'score'], ascending=False)
print(df_separation)

        problem_id  possible_user     score
254072         379           4018  0.009468
223108         322           4018  0.007531
309110         489           4018  0.007359
297129         471           4018  0.007144
291194         468           4018  0.007134
78107          109           4018  0.007130
275426         441           4018  0.007101
244087         355           4018  0.007091
242092         352           4018  0.007078
238152         346           4018  0.007074
287226         465           4018  0.007070
240118         349           4018  0.007069
213303         312           4018  0.007060
232253         336           4018  0.007060
209426         310           4018  0.007058
32057           39           4018  0.007048
1640             2           4018  0.007030
100657         143           4018  0.007030
236194         342           4018  0.007029
259872         390           4018  0.007026
89762          128           4018  0.007026
79966          114           401

In [156]:
# ahora tengo que hacer un nuevo dataframe con usuario y problema, y una lista de recommendation 

# hago primero la agrupacion por usuario
grouped_r = df_separation.groupby('possible_user')

# hago la agregacion en una lista 
df_recommend = grouped_r.aggregate(lambda x:list(x))

#print(df_recommend)

# vuelvo a crear la estructura de los datos para poder trabajar con ellos
df_recommend = pd.DataFrame({'user_id':df_recommend.index.values, 'problem_id':df_recommend['problem_id'].tolist()})
print(df_recommend)

                                             problem_id  user_id
0     [322, 269, 93, 17, 166, 60, 312, 262, 222, 29,...        5
1     [485, 440, 373, 469, 358, 311, 282, 266, 272, ...        6
2     [128, 2, 143, 312, 310, 327, 100, 254, 256, 25...        7
3     [322, 231, 35, 239, 65, 254, 100, 49, 171, 145...        8
4     [322, 379, 128, 114, 39, 141, 155, 159, 203, 2...        9
5     [322, 489, 114, 436, 308, 312, 379, 310, 2, 14...       10
6     [489, 379, 114, 128, 2, 143, 254, 256, 258, 43...       11
7     [88, 90, 191, 251, 10, 23, 181, 174, 29, 313, ...       12
8     [114, 436, 441, 2, 143, 100, 254, 256, 258, 12...       15
9     [322, 489, 114, 436, 2, 143, 100, 254, 256, 25...       16
10    [322, 489, 114, 2, 143, 436, 100, 254, 256, 25...       17
11    [467, 279, 282, 446, 470, 311, 340, 128, 510, ...       18
12    [183, 139, 237, 136, 181, 191, 325, 438, 130, ...       22
13    [322, 379, 254, 256, 258, 436, 39, 114, 249, 4...       24
14    [308, 489, 503, 307

In [157]:
def getProblems(row, df_recommend):
    """
        Funcion que copia los posibles problemas a recomendar para los usuarios a los que tengo que recomendar
    """
    
    # saco la lista de problemas 
    df_lista_problemas_original = df_recommend[df_recommend['user_id'] == row['user_id']]
    lista_problemas_original = list(df_lista_problemas_original['problem_id'])[0]
    
    return lista_problemas_original
    

In [158]:
# df_users # en este dataframe estan los usuarios a los que quiero recomendar

# ahora voy a copiar los problemas para los usuarios a los que tengo que recomendar
df_users['recommendation'] = df_users.apply (lambda row: getProblems(row, df_recommend), axis=1)

print(df_users)

                                        list_problem_id  user_id  \
11                            [119, 114, 316, 507, 506]       18   
14    [258, 141, 309, 310, 390, 187, 510, 438, 13, 2...       25   
39    [100, 62, 2, 57, 53, 23, 134, 15, 147, 233, 24...       60   
90    [309, 150, 183, 237, 191, 187, 39, 209, 70, 15...      130   
241   [213, 2, 109, 114, 10, 436, 438, 437, 404, 239...      414   
255   [39, 100, 247, 150, 183, 44, 471, 109, 307, 30...      443   
257   [255, 257, 314, 311, 315, 310, 282, 243, 275, ...      448   
392   [171, 272, 282, 209, 373, 155, 2, 39, 134, 471...      689   
512   [241, 269, 2, 340, 65, 379, 316, 275, 309, 259...      912   
523   [65, 241, 239, 105, 231, 269, 275, 331, 257, 1...      935   
933   [507, 33, 83, 217, 312, 256, 49, 4, 503, 254, ...     1711   
997                      [39, 49, 465, 159, 134, 13, 2]     1893   
1021  [325, 441, 237, 183, 438, 510, 226, 4, 327, 21...     1952   
1024      [325, 327, 441, 468, 471, 503, 226, 10

In [159]:
# elimino la columna de los problemas, ya que no interesa
del df_users['list_problem_id']

print(df_users)

      user_id                                     recommendation
11         18  [467, 279, 282, 446, 470, 311, 340, 128, 510, ...
14         25  [308, 489, 503, 307, 468, 315, 322, 346, 436, ...
39         60  [379, 508, 171, 255, 256, 258, 310, 109, 114, ...
90        130  [128, 255, 247, 233, 114, 249, 49, 254, 256, 2...
241       414  [128, 105, 57, 247, 159, 78, 86, 233, 256, 100...
255       443  [379, 346, 128, 256, 436, 143, 162, 308, 243, ...
257       448  [489, 467, 470, 365, 272, 507, 469, 486, 93, 4...
392       689  [279, 489, 128, 143, 78, 86, 203, 233, 247, 10...
512       912  [322, 489, 467, 358, 440, 373, 282, 470, 279, ...
523       935  [39, 4, 145, 336, 342, 44, 183, 53, 13, 437, 3...
933      1711  [259, 309, 308, 437, 251, 231, 352, 53, 374, 3...
997      1893  [322, 379, 109, 143, 114, 128, 100, 254, 256, ...
1021     1952  [379, 467, 465, 471, 468, 506, 139, 390, 312, ...
1024     1955  [379, 465, 390, 114, 39, 308, 445, 508, 336, 3...
1066     2025  [313, 465,

In [160]:
def getKrecomFinal(row, k):
    """
        Funcion que saca las k mejores recomendaciones para el usuario
        Lo que hace es coger los primeros k valores de la lista de recomendaciones
    """
    return row['recommendation'][:k]

In [161]:
k = 10

#print(list(range(1, k+1)))

rango = list(range(1, k+1))

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    #print(name_column)
    # ahora saco los k mejores problemas para cada usuario
    df_users[name_column] = df_users.apply(lambda row: getKrecomFinal(row, i), axis=1)

# he incluido una columna para cada cada k recomendacion 
print(df_users)

      user_id                                     recommendation  \
11         18  [467, 279, 282, 446, 470, 311, 340, 128, 510, ...   
14         25  [308, 489, 503, 307, 468, 315, 322, 346, 436, ...   
39         60  [379, 508, 171, 255, 256, 258, 310, 109, 114, ...   
90        130  [128, 255, 247, 233, 114, 249, 49, 254, 256, 2...   
241       414  [128, 105, 57, 247, 159, 78, 86, 233, 256, 100...   
255       443  [379, 346, 128, 256, 436, 143, 162, 308, 243, ...   
257       448  [489, 467, 470, 365, 272, 507, 469, 486, 93, 4...   
392       689  [279, 489, 128, 143, 78, 86, 203, 233, 247, 10...   
512       912  [322, 489, 467, 358, 440, 373, 282, 470, 279, ...   
523       935  [39, 4, 145, 336, 342, 44, 183, 53, 13, 437, 3...   
933      1711  [259, 309, 308, 437, 251, 231, 352, 53, 374, 3...   
997      1893  [322, 379, 109, 143, 114, 128, 100, 254, 256, ...   
1021     1952  [379, 467, 465, 471, 468, 506, 139, 390, 312, ...   
1024     1955  [379, 465, 390, 114, 39, 308, 445

In [162]:
# elimino las columnas que no me interesan
del df_users['recommendation']

print(df_users)

      user_id k_recommendation_1 k_recommendation_2 k_recommendation_3  \
11         18              [467]         [467, 279]    [467, 279, 282]   
14         25              [308]         [308, 489]    [308, 489, 503]   
39         60              [379]         [379, 508]    [379, 508, 171]   
90        130              [128]         [128, 255]    [128, 255, 247]   
241       414              [128]         [128, 105]     [128, 105, 57]   
255       443              [379]         [379, 346]    [379, 346, 128]   
257       448              [489]         [489, 467]    [489, 467, 470]   
392       689              [279]         [279, 489]    [279, 489, 128]   
512       912              [322]         [322, 489]    [322, 489, 467]   
523       935               [39]            [39, 4]       [39, 4, 145]   
933      1711              [259]         [259, 309]    [259, 309, 308]   
997      1893              [322]         [322, 379]    [322, 379, 109]   
1021     1952              [379]      

In [163]:
# ahora tengo que filtrar df_users_eval para que solo contenga las filas de los usuarios a los que hay que recomendar

df_users_eval_filter = df_users_eval[df_users_eval['user_id'].isin(user_list_to_recommend)]
print(df_users_eval_filter)

                                       list_problem_id  user_id
4               [241, 128, 124, 256, 404, 469, 70, 86]       18
5                    [33, 29, 254, 308, 445, 253, 166]       25
8    [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...       60
13                       [254, 253, 308, 166, 49, 445]      130
22   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...      414
26             [134, 187, 256, 251, 90, 174, 147, 438]      443
27   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...      448
34                     [327, 23, 150, 349, 336, 6, 49]      689
44   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...      912
46          [95, 383, 262, 122, 44, 81, 252, 145, 336]      935
70   [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...     1711
73                       [471, 469, 355, 95, 243, 436]     1893
78         [374, 195, 109, 436, 95, 10, 446, 340, 379]     1952
80                            [44, 213, 136, 162, 243]     1955
90             [136, 253, 187, 342, 203,

In [164]:
list_eval_problems = df_users_eval_filter['list_problem_id'].tolist()


# meto toda la informacion en un dataframe para obtener las metricas
set_df_metric = {'user_id': user_list_to_recommend, 'eval_problems': list_eval_problems}

for i in rango:
    name_column = 'k_recommendation_' + str(i)
    list_recom_problems = list()
    list_recom_problems = df_users[name_column].tolist()
    
    name_column_metric = 'recom_problems_' + str(i)
    set_df_metric[name_column_metric] = list_recom_problems

# he generado un nuevo dataframe con todas las recomendaciones (desde 1 a k) y los problemas realizados en 
# el evaluation set para cada usuario al que quiero recomendar

metric_df = pd.DataFrame.from_dict(set_df_metric)

print(metric_df)

                                        eval_problems recom_problems_1  \
0              [241, 128, 124, 256, 404, 469, 70, 86]            [467]   
1                   [33, 29, 254, 308, 445, 253, 166]            [308]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...            [379]   
3                       [254, 253, 308, 166, 49, 445]            [128]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...            [128]   
5             [134, 187, 256, 251, 90, 174, 147, 438]            [379]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...            [489]   
7                     [327, 23, 150, 349, 336, 6, 49]            [279]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...            [322]   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]             [39]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...            [259]   
11                      [471, 469, 355, 95, 243, 436]            [322]   
12        [374, 195, 109, 436, 95, 10,

In [165]:
def one_hit(row, i):
    """
        Funcion que implementa la metrica one hit. Devuelve un 1 si para un usuarios dado, al menos uno
        de los problemas que se le ha recomendado ha sido realizado por ese usuario en el evaluation_set. 
        Cero si no hay ningun problema de los recomendados que haya sido realizado por el usuario
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:
        return 1
    else:
        return 0

In [166]:
def mrr(row, i): 
    """
        Funcion que va a implementar la metrica de evaluacion mrr:
        mrr = 1/ranki, donde ranki es la posicion del primer item correcto
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    if len(num_problems_common) >= 1:

        # hago la busqueda del primer elemento que esta en la lista de recomendados
        fst_correct_item = -1
        encontrado = False
        i = 0
        while (i < len(row[name_column])) and (encontrado == False):
            if row[name_column][i] in row['eval_problems']:
                # fst_correct_item = row['recom_problems'][i]
                # print(fst_correct_item)
                ranki = i + 1
                encontrado = True
            else:
                i = i + 1
                
        return (1/ranki)

    else:
        return 0

In [167]:
def precision(row, i):
    """
        Funcion que va a implementar la metrica precision en k: 
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los recomendados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row[name_column]))

In [168]:
def recall(row, i):
    """
        Funcion que implementa la metrica recall
        (cuantos de los realizados por el usuario estan entre los recomendados) / todos los evaluados
    """
    name_column = 'recom_problems_' + str(i)
    num_problems_common = np.intersect1d(row[name_column], row['eval_problems'])
    
    # print(num_problems_common)
    
    return (len(num_problems_common)/len(row['eval_problems']))

In [169]:
def f1(row, i):
    """
        Funcion que calcula el f1 en funcion de precision y recall
    """
    name_column_prec = 'precision_' + str(i)
    name_column_rec = 'recall_' + str(i)
    denominador = row[name_column_prec] + row[name_column_rec]
    
    if denominador == 0:
        return 0
    else:
        return (2 * row[name_column_prec] * row[name_column_rec]) / denominador

In [170]:
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    # ahora voy a calcular una metrica para cada usuario
    metric_df[name_one_hit] = metric_df.apply(lambda row: one_hit(row, i), axis=1)
    metric_df[name_mrr] = metric_df.apply(lambda row: mrr(row, i), axis=1)
    metric_df[name_precision] = metric_df.apply(lambda row: precision(row, i), axis=1)
    metric_df[name_recall] = metric_df.apply(lambda row: recall(row, i), axis=1)
    metric_df[name_f1] = metric_df.apply(lambda row: f1(row, i), axis=1)

print(metric_df)

                                        eval_problems recom_problems_1  \
0              [241, 128, 124, 256, 404, 469, 70, 86]            [467]   
1                   [33, 29, 254, 308, 445, 253, 166]            [308]   
2   [44, 228, 141, 217, 251, 105, 4, 155, 181, 191...            [379]   
3                       [254, 253, 308, 166, 49, 445]            [128]   
4   [39, 393, 390, 249, 233, 25, 27, 29, 35, 53, 5...            [128]   
5             [134, 187, 256, 251, 90, 174, 147, 438]            [379]   
6   [203, 136, 141, 2, 307, 325, 336, 312, 313, 31...            [489]   
7                     [327, 23, 150, 349, 336, 6, 49]            [279]   
8   [128, 247, 44, 81, 272, 279, 282, 322, 331, 37...            [322]   
9          [95, 383, 262, 122, 44, 81, 252, 145, 336]             [39]   
10  [224, 251, 259, 485, 93, 437, 53, 166, 374, 44...            [259]   
11                      [471, 469, 355, 95, 243, 436]            [322]   
12        [374, 195, 109, 436, 95, 10,

In [171]:


f = open("C:/hlocal/TFM/vot_simple", 'a')

# calculo la media de las metricas
for i in rango:
    name_one_hit = 'one_hit_' + str(i)
    name_mrr = 'mrr_' + str(i)
    name_precision = 'precision_' + str(i)
    name_recall = 'recall_' + str(i)
    name_f1 = 'f1_' + str(i)
    
    result_one_hit = metric_df[name_one_hit].mean()
    result_precision = metric_df[name_precision].mean()
    result_mrr = metric_df[name_mrr].mean()
    result_recall = metric_df[name_recall].mean()
    result_f1 = metric_df[name_f1].mean()

    # lo muestro por consola
    
    print(i)
    print("###########")
    print("One hit ----------")
    print(result_one_hit)
    print("Precision ----------")
    print(result_precision)
    print("Mrr  ----------")
    print(result_mrr)
    print("Recall  ----------")
    print(result_recall)
    print("F1  ----------")
    print(result_f1)
    
    
    f.write(str(result_one_hit) + '\t' + str(result_precision) + '\t' + str(result_mrr) + '\t' + str(result_recall) + '\t' +  str(result_f1) + '\n') 

    
f.close()    
    

1
###########
One hit ----------
0.36923076923076925
Precision ----------
0.36923076923076925
Mrr  ----------
0.36923076923076925
Recall  ----------
0.04367927747647354
F1  ----------
0.07679440663457594
2
###########
One hit ----------
0.4307692307692308
Precision ----------
0.25384615384615383
Mrr  ----------
0.4
Recall  ----------
0.05653623624601297
F1  ----------
0.08937173302736531
3
###########
One hit ----------
0.5538461538461539
Precision ----------
0.25641025641025644
Mrr  ----------
0.44102564102564107
Recall  ----------
0.0881112291182019
F1  ----------
0.12568989515223636
4
###########
One hit ----------
0.6615384615384615
Precision ----------
0.2653846153846154
Mrr  ----------
0.46794871794871795
Recall  ----------
0.1220173674663327
F1  ----------
0.15923273796934193
5
###########
One hit ----------
0.676923076923077
Precision ----------
0.2430769230769231
Mrr  ----------
0.47102564102564104
Recall  ----------
0.14142573687470208
F1  ----------
0.1697013220484247
6
####